# Assignment 7
## Lukas Hager
___

In [1]:
import numpy as np

There are eight distinct choice probabilities, one for each of the eight possible states, given by $(a_{it-1}, a_{-it-1}, M_t)$. In plain English, the possibilities are all combinations of your past action, your opponent's past action, and the market forces.

Following the notation of the notes, we can write in matrix form that for $8\times1$ value function $V_i^*$ induced by an equilibrium choice probability vector $P_i^*$ as 
$$\left(I-\beta F^{P^*}\right)V_i^{P^*} = \sum_{a_i=0}^1P_i^*(a_i) \times \left(\pi_i^{P^*}(a_i) + e_i^{P^*}(a_i)\right)$$
Here, $F^{P^*}$ is the transition matrix induced by $P^*$, $\pi_i^{P^*}(a_i)$ is the expected payoff of playing strategy $a_i$ in state $x$, and $e_i^{P^*}(a_i)$ is
$$\mathbb{E}\left[\varepsilon(a_i)|\sigma^*=a_i,x\right]$$
To calculate this final term, we use the properties of the Gumbel distribution:
$$\mathbb{E}\left[\varepsilon(a_i)|x,\sigma^* = a_i\right] = \mu + \log\left(1 + \exp(\pi(a_{-i},x) - \pi(a_{i},x))\right)$$

In [2]:
beta = .95
lam = 2
delta = 2
gamma = 1.5
mu = .5772
M = np.array([1.,1.5])
B = np.array([.5,.5,.4,.6]).reshape(2,-1)
X = (np.array(np.meshgrid([0,1], [0,1], [1,1.5])).T).reshape(-1,3)
X = X[np.lexsort((X[:,1], X[:,2]))]

In [3]:
# payoff function
def Pi(choice, a_i, a_j, M):
    return(choice * (lam * M - delta * a_j - gamma * (1-a_i)))

# expected payoff
def pi(choice, P_cond):
    a_i = X[:,0]
    M = X[:,2]
    
    P_cond_2 = P_cond.copy()
    P_cond_2[2], P_cond_2[1], P_cond_2[6], P_cond_2[5] = P_cond_2[1], P_cond_2[2], P_cond_2[5], P_cond_2[6]
    
    Pi_val = Pi(choice, a_i, np.ones(8), M) * P_cond_2 + Pi(choice, a_i, np.zeros(8), M) * (1.-P_cond_2)
    return(Pi_val)

# conditional gumbel expectation
def e(choice, P_cond):
    other = 1. - choice
    return(mu + np.log(1. + np.exp(pi(other,P_cond) - pi(choice,P_cond))))

In [4]:
# create transition probability function for given P
# order: (m1,0,0), (m1,1,0), (m1,0,1), (m1,1,1), (m2,0,0), (m2,1,0), (m2,0,1), (m2,1,1)

def F(P_cond):
    # flip indices for second player
    P_cond_2 = P_cond.copy()
    P_cond_2[2], P_cond_2[1], P_cond_2[6], P_cond_2[5] = P_cond_2[1], P_cond_2[2], P_cond_2[5], P_cond_2[6]
    
    P_1 = np.column_stack([1.-P_cond, 
                           P_cond, 
                           1.-P_cond, 
                           P_cond])
    P_2 = np.column_stack([1.-P_cond_2, 
                           1.-P_cond_2, 
                           P_cond_2, 
                           P_cond_2])
    big_P_1 = np.column_stack([P_1,P_1])
    big_P_2 = np.column_stack([P_2,P_2])
    
    big_B = np.repeat(np.repeat(B, 4, axis = 0), 4, axis = 1)

    return(big_P_1 * big_P_2 * big_B)

In [5]:
def Gamma(P_cond):
    inv_term = np.linalg.inv(np.eye(8) - beta * F(P_cond))
    e_1 = e(1., P_cond)
    e_0 = e(0., P_cond)
    sum_term = P_cond * (pi(1., P_cond) + e_1) + (1.-P_cond) * e_0
    return(inv_term @ sum_term.reshape(-1,1))

In [6]:
def Psi(P_cond):
    gamma_vals = Gamma(P_cond)
    probs = F(P_cond)
    
    in_probs = probs.copy()
    in_probs[:,np.where(X[:,0] == 0.)] = 0.
    
    out_probs = probs.copy()
    out_probs[:,np.where(X[:,0] == 1.)] = 0.
    
    cond_prob_in = in_probs / np.sum(in_probs, axis = 1).reshape(-1,1)
    cond_prob_out = out_probs / np.sum(out_probs, axis = 1).reshape(-1,1)
    
    e_1 = e(1., P_cond).reshape(-1,1)
    e_0 = e(0., P_cond).reshape(-1,1)
        
    V_1 = pi(1., P_cond).reshape(-1,1) + e_1 + beta * cond_prob_in @ gamma_vals
    V_0 = pi(0., P_cond).reshape(-1,1) + e_0 + beta * cond_prob_out @ gamma_vals
        
    P_update = np.exp(V_1) / (np.exp(V_0) + np.exp(V_1))
    
    
    return(P_update.reshape(-1))

In [7]:
converged = False
crit = 1e-6
P_cond_old = np.ones(8) / 2.
while not converged:
    P_cond_new = Psi(P_cond_old)
    if np.max(np.abs(P_cond_new - P_cond_old)) <= crit:
        converged = True
    else:
        P_cond_old = P_cond_new

Thus, our equilibrium solution $P^*$ of choice probabilities is as follows:

In [8]:
P_cond_new

array([0.69539266, 0.69539266, 0.69539266, 0.69539266, 0.70216229,
       0.70216229, 0.70216229, 0.70216229])

where the states are given as tuples of the form $(a_{it-1},a_{-it-1},M)$ by

In [9]:
X

array([[0. , 0. , 1. ],
       [1. , 0. , 1. ],
       [0. , 1. , 1. ],
       [1. , 1. , 1. ],
       [0. , 0. , 1.5],
       [1. , 0. , 1.5],
       [0. , 1. , 1.5],
       [1. , 1. , 1.5]])